This notebook is meant for our lecture 5 breakout session.  I've put together some questions to guide your [here](https://itempool.com/MIT-Robotic-Manipulation/c/uMW2Pmz_fPa).  The points are not real, and will not contribute to your grade!

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='9eef22b4c0c66268debe69adef12ce140a55ca05', drake_version='20200918', drake_build='nightly')

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Let's do all of our imports here, too.
import numpy as np
from functools import partial
from IPython.display import display
from ipywidgets import FloatSlider, Textarea, Layout

from pydrake.all import (DiagramBuilder, AddMultibodyPlantSceneGraph, RigidTransform, ConnectMeshcatVisualizer,
                         JacobianWrtVariable, Parser, FixedOffsetFrame,
                         PiecewisePolynomial, MathematicalProgram, LinearConstraint, Solve)
from manipulation.meshcat_utils import plot_mathematical_program
from manipulation.utils import FindResource


# This one is specific to this notebook, but I'm putting it in the header to make it less distracting.
def Visualizer(MakeMathematicalProgram):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    twolink = Parser(plant, scene_graph).AddModelFromFile(
        FindResource("models/double_pendulum.urdf"))
    hand = plant.AddFrame(frame=FixedOffsetFrame(
        name="hand", P=plant.GetFrameByName("lower_arm"),
        X_PF=RigidTransform([0, 0, -1])))
    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(builder,
                                  scene_graph,
                                  zmq_url=zmq_url)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    meshcat.vis.delete()
    #meshcat.vis["/Background"].set_property('visible',False)
    meshcat.vis["/Background"].set_property('top_color', [0, 0, 0])
    meshcat.vis["/Background"].set_property('bottom_color', [0, 0, 0])
    meshcat.vis["/Grid"].set_property('visible',False)
    meshcat.load()

    jacobian = Textarea(value="", description="J_G: ", layout={'width':'200pm','height':'50px'}, style={'description_width':'initial'})
    display(jacobian)

    X, Y = np.meshgrid(np.linspace(-5, 5, 35), np.linspace(-5, 5, 31))

    def visualize(q, v_Gdesired=[1.0, 0.0], t=None):
        if t:
            context.SetTime(t)
        plant.SetPositions(plant_context, q)
        diagram.Publish(context)

        J_G = plant.CalcJacobianTranslationalVelocity(plant_context, JacobianWrtVariable.kQDot, hand, [0,0,0], plant.world_frame(), plant.world_frame())
        J_G = J_G[[0,2],:]  # Ignore Y.
        jacobian.value = np.array2string(J_G, formatter={'float': lambda x: "{:5.2f}".format(x)})

        prog = MakeMathematicalProgram(q, J_G, v_Gdesired)
        result = Solve(prog)
        v = meshcat.vis["QP"]
        plot_mathematical_program(v, prog, X, Y, result=result)
        # TODO: Add set_object to meshcat.Animation
        if False: # meshcat._is_recording:
            with meshcat._animation.at_frame(
                    v, meshcat._recording_frame_num) as m:
                plot_mathematical_program(m, prog, X, Y, result=result)

    return visualize, meshcat

# Differential Inverse Kinematics as a Quadratic Program

## Define your mathematical program here.


In [ ]:
def MakeMathematicalProgram(q, J_G, v_Gdesired):
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(2, 'v')
    v_max = 3.0 

    error = J_G.dot(v) - v_Gdesired
    prog.AddCost(error.dot(error))
    prog.AddBoundingBoxConstraint(-v_max, v_max, v)

    return prog

## Visualize a particular joint angle

In [ ]:
visualize, meshcat = Visualizer(MakeMathematicalProgram)

q = [-np.pi/2.0+0.5, -1.0]
v_Gdesired = [0.5, 0.]
visualize(q, v_Gdesired)

## Animated joint trajectory (passing through the singularity)

In [ ]:
import time

visualize, meshcat = Visualizer(MakeMathematicalProgram)

v_Gdesired = [1.0, 0.0]
T = 2.
q = PiecewisePolynomial.FirstOrderHold(
    [0, T, 2*T], np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1., -np.pi / 2.0 + 1.], 
                           [-2., 2., -2]]))

nx = 35
ny = 31
X, Y = np.meshgrid(np.linspace(-5, 5, nx), np.linspace(-5, 5, ny))
D = np.vstack((X.reshape(1,-1), Y.reshape(1,-1)))
for i in range(2):
    for t in np.linspace(0, 2*T, num=100):
        visualize(q.value(t), v_Gdesired, t=t)
        time.sleep(0.05)

## Trajectory slider

In [ ]:
visualize, meshcat = Visualizer(MakeMathematicalProgram)

v_Gdesired = [1.0, 0.0]
T = 2.
qtraj = PiecewisePolynomial.FirstOrderHold(
    [0, T], np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1.], [-2., 2.]]))
visualize(qtraj.value(0), v_Gdesired)

def _t_callback(change):
    visualize(qtraj.value(change.new), v_Gdesired)

slider = FloatSlider(value=0, min=0, max=T, step=0.05, continuous_update=True, description="t", layout=Layout(width="'100'"))
slider.observe(_t_callback, names='value')
display(slider)


## Joint Sliders

In [ ]:
visualize, meshcat = Visualizer(MakeMathematicalProgram)

q = [-np.pi/2.0 + 0.5, -1.0]
v_Gdeised = [1.0, 0.0]
visualize(q, v_Gdesired)

def _q_callback(change, index):
    q[index] = change.new
    visualize(q, v_Gdesired)
def _vG_callback(change, index):
    v_Gdesired[index] = change.new
    visualize(q, v_Gdesired)

slider = FloatSlider(value=q[0], min=-np.pi, max=np.pi, step=0.1, continuous_update=True, description="q0", layout=Layout(width="'100'"))
slider.observe(partial(_q_callback, index=0), names='value')
display(slider)

slider = FloatSlider(value=q[1], min=-np.pi, max=np.pi, step=0.1, continuous_update=True, description="q1", layout=Layout(width="'100'"))
slider.observe(partial(_q_callback, index=1), names='value')
display(slider)

slider = FloatSlider(value=v_Gdesired[0], min=-4, max=4, step=0.1, continuous_update=True, description="v_G_W0", layout=Layout(width="'100'"))
slider.observe(partial(_vG_callback, index=0), names='value')
display(slider)

slider = FloatSlider(value=v_Gdesired[1], min=-4, max=4, step=0.1, continuous_update=True, description="v_G_W1", layout=Layout(width="'100'"))
slider.observe(partial(_vG_callback, index=1), names='value')
display(slider)